In [5]:
#!/usr/bin/env python
# coding: utf-8

bucket = 'czambrano-sagemaker' 
prefix = 'sagemaker/xgboost_credit_risk'

In [6]:
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
import csv
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

In [7]:
bucket

'czambrano-sagemaker'

In [8]:
prefix

'sagemaker/xgboost_credit_risk'

In [9]:
role = get_execution_role() 


In [10]:
role

'arn:aws:iam::132038315038:role/service-role/AmazonSageMaker-ExecutionRole-20201210T161518'

In [11]:
get_ipython().system('wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls')


--2021-01-10 23:33:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  8.62MB/s    in 0.6s    

2021-01-10 23:33:05 (8.62 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



In [12]:
dataset = pd.read_excel('default of credit card clients.xls')


In [13]:
pd.set_option('display.max_rows', 8)


In [14]:
pd.set_option('display.max_columns', 15)


In [15]:
dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,...,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,...,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,...,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,29997,150000,1,3,2,43,-1,...,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,...,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,...,85900,3409,1178,1926,52964,1804,1
30000,30000,50000,1,2,1,46,0,...,2078,1800,1430,1000,1000,1000,1


In [16]:
dataset = dataset.drop('Unnamed: 0', axis=1)


In [17]:
dataset


,X1,X2,X3,X4,X5,X6,X7,...,X18,X19,X20,X21,X22,X23,Y
0,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,...,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,...,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,150000,1,3,2,43,-1,-1,...,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,...,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,...,85900,3409,1178,1926,52964,1804,1
30000,50000,1,2,1,46,0,0,...,2078,1800,1430,1000,1000,1000,1


In [18]:
dataset = pd.concat([dataset['Y'], dataset.drop(['Y'], axis=1)], axis=1)


In [19]:
dataset

,Y,X1,X2,X3,X4,X5,X6,...,X17,X18,X19,X20,X21,X22,X23
0,default payment next month,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
1,1,20000,2,2,1,24,2,...,0,0,689,0,0,0,0
2,1,120000,2,2,2,26,-1,...,3261,0,1000,1000,1000,0,2000
3,0,90000,2,2,2,34,0,...,15549,1518,1500,1000,1000,1000,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,0,150000,1,3,2,43,-1,...,0,1837,3526,8998,129,0,0
29998,1,30000,1,2,2,37,4,...,19357,0,0,22000,4200,2000,3100
29999,1,80000,1,3,1,41,1,...,48944,85900,3409,1178,1926,52964,1804
30000,1,50000,1,2,1,46,0,...,15313,2078,1800,1430,1000,1000,1000


In [20]:
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])


In [21]:
train_data.to_csv('train.csv', header=False, index=False)


In [22]:
train_data


,Y,X1,X2,X3,X4,X5,X6,...,X17,X18,X19,X20,X21,X22,X23
26386,0,240000,2,1,2,29,-1,...,696,700,696,18334,10000,696,696
11207,1,200000,1,2,1,28,2,...,2443,0,0,0,0,0,0
23077,0,200000,2,1,2,32,-1,...,16890,3904,24277,4996,0,16890,6077
24734,0,90000,1,1,1,34,0,...,0,20585,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,0,170000,1,2,1,40,-1,...,2400,3409,6530,2578,2100,2400,6665
9883,0,70000,2,3,1,47,1,...,28976,3107,48323,0,2001,3000,3003
16909,1,180000,1,2,1,42,2,...,31274,1452,1526,1432,1200,1072,1326
2956,0,250000,2,1,2,37,0,...,17793,1007,13346,9163,8338,18618,1763


In [23]:
validation_data.to_csv('validation.csv', header=False, index=False)


In [24]:
validation_data

,Y,X1,X2,X3,X4,X5,X6,...,X17,X18,X19,X20,X21,X22,X23
14913,1,330000,2,2,1,42,-2,...,500,15450,0,209,3,503,0
29986,0,240000,1,1,2,30,-2,...,0,0,0,0,0,0,0
21366,0,20000,1,2,2,54,0,...,11710,1200,1200,1600,1500,1700,1000
1261,0,150000,1,1,2,37,0,...,52549,3000,2500,3000,3000,1546,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24863,0,70000,1,2,1,37,0,...,48525,2538,2607,2137,1714,1751,2153
15283,0,240000,2,1,2,28,0,...,199259,7200,7500,6300,6000,6100,6000
19791,0,520000,2,1,2,35,0,...,54152,5000,7000,5000,5004,5000,5000
18847,0,130000,2,2,1,28,0,...,83234,6500,6600,6000,3100,3200,3300


In [25]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

NameError: name 'TrainingInput' is not defined

In [26]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')


In [27]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')


In [28]:
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')


NameError: name 'TrainingInput' is not defined

In [29]:
from sagemaker.inputs import TrainingInput

In [30]:
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [31]:
s3_input_train

In [32]:
s3_input_validation

In [33]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

In [34]:
containers

{'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
 'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
 'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
 'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

In [35]:
sess = sagemaker.Session()

In [36]:
xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

In [37]:
xbg

NameError: name 'xbg' is not defined

In [38]:
xgb.set_hyperparameters(eta=0.1, objective='binary:logistic', num_round=25) 


In [39]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})


2021-01-11 00:15:42 Starting - Starting the training job...
2021-01-11 00:16:05 Starting - Launching requested ML instancesProfilerReport-1610324142: InProgress
......
2021-01-11 00:17:06 Starting - Preparing the instances for training......
2021-01-11 00:18:07 Downloading - Downloading input data...
2021-01-11 00:18:32 Training - Downloading the training image..Arguments: train
[2021-01-11:00:18:53:INFO] Running standalone xgboost training.
[2021-01-11:00:18:53:INFO] File size need to be processed in the node: 2.32mb. Available memory size in the node: 8429.18mb
[2021-01-11:00:18:53:INFO] Determined delimiter of CSV input is ','
[00:18:53] S3DistributionType set as FullyReplicated
[00:18:53] 21000x23 matrix with 483000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-01-11:00:18:53:INFO] Determined delimiter of CSV input is ','
[00:18:53] S3DistributionType set as FullyReplicated
[00:18:53] 6000x23 matrix with 138000 entries loaded from /opt/ml/

In [42]:
xgb_predictor = xgb.deploy(
	initial_instance_count = 1,
	instance_type = 'ml.m4.xlarge',
	serializer = CSVSerializer())

---------------!

In [41]:
from sagemaker.serializers import CSVSerializer

In [43]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.to_numpy()[:,1:])

In [44]:
predictions


array([0.10145303, 0.29316297, 0.70782304, ..., 0.15233986, 0.38610485,
       0.10773233])